<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Tradevesting/myProspects_Upside_Analysis_ATH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import datetime
import pytz
def get_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

get_time()
gen_output = 0

Run date time (IST): 2023-10-20 11:59:24


In [2]:
# install libraries
!pip install yfinance
!pip install pandas-ta
#!pip install talib-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=7e21529e260ec31a5e7c78d0d4269af2d7a391ba82c87fd3f39ee3aa37348b5c
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [3]:
# set start and end date
import datetime
start_date = (datetime.date.today() + datetime.timedelta(days=-365*5)).strftime('%Y-%m-%d')
start1_date = (datetime.date.today() + datetime.timedelta(days=-1*365)).strftime('%Y-%m-%d')
end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

In [4]:
# fetch nifty200 scrips

import pandas as pd
import numpy as np

def read_file(path):
 df = pd.read_csv(path)
 return df

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Tradevesting/'

mypfs = pd.read_csv(path_mypf + 'myPortfolioStocks.csv')
mypps = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

df_mypf = pd.merge(mypfs, mypps, on="Symbol")


def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'ADANITRANS', 'SREERAYALK', 'SCHAEFFLER ']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

In [5]:
df_mypf = df_mypf[df_mypf['InPortfolio'].isnull()]
cols = ['Symbol', 'Criteria', 'Strategy', 'LatestQtr', 'Target','StarStock', 'BizCheck', 'FinCheck', 'TechCheck', 'Conviction', 'Category', 'Stock']
df_mypf = df_mypf[cols]
all_prospects = get_stock_ids(df_mypf)

In [6]:
def print_distribution():

  # star stocks
  print('Star Prospects:')
  print(df_mypf['StarStock'].value_counts())
  print('-------')

  # v40/v40n stocks
  print('V40/V40N Prospects:')
  print(df_mypf['Criteria'].value_counts())
  print('-------')

  # latest quarter
  print('Latest Qtr:')
  print(df_mypf['LatestQtr'].value_counts())
  print('-------')

  # star + v40/v40n stocks
  print('Star + V40/V40N Stocks:')
  print(df_mypf[(df_mypf['Criteria'] == 'V40') | (df_mypf['Criteria'] == 'V40N')]['StarStock'].value_counts())
  print('-------')

print_distribution()

Star Prospects:
0    68
1    64
Name: StarStock, dtype: int64
-------
V40/V40N Prospects:
V40N    28
V40     23
VR       5
AR       4
V200     1
Name: Criteria, dtype: int64
-------
Latest Qtr:
1    122
0     10
Name: LatestQtr, dtype: int64
-------
Star + V40/V40N Stocks:
1    27
0    24
Name: StarStock, dtype: int64
-------


In [7]:
# import libraries

%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib
yfin.pdr_override()
import pandas_ta as ta

from google.colab import data_table

In [8]:
# fetch current PE
import requests
from bs4 import BeautifulSoup

def get_current_features(stock_name):

  link_blueprint = "https://www.screener.in/company/"

  stocks = ['INDIGOPNTS', 'IRCTC', 'GILLETTE', 'PAGEIND', 'BANDHANBNK', 'ABBOTINDIA', 'JCHAC',
              'NESTLEIND', 'PGHH', 'EQUITASBNK', 'SBICARD', 'ASTRAZEN', 'ICICIGI', 'RELAXO']

  if stock_name in stocks:
    actual_link = link_blueprint + stock_name.upper()
  else:
    actual_link = link_blueprint + stock_name.upper() + "/consolidated/"

  page = requests.get(actual_link)
  #print(actual_link)
  link_soup = BeautifulSoup(page.content, 'html.parser')

  test_list = link_soup.find(id="top").text.replace("  ", "").split('\n')

  while("" in test_list) :
      test_list.remove("")
  while("\r" in test_list) :
      test_list.remove("\r")
  while(" \r" in test_list) :
      test_list.remove(" \r")

  try:

    current_pe = float(test_list[test_list.index('Stock P/E')+1:test_list.index('Stock P/E')+2][0])
    current_roce = float(test_list[test_list.index('ROCE')+1:test_list.index('ROCE')+2][0])
    current_roe = float(test_list[test_list.index('ROE')+1:test_list.index('ROE')+2][0])
  except ValueError as ve:
    current_pe = 1000
    current_roce = 1000
    current_roe = 1000

  return current_pe, current_roce, current_roe

In [9]:
# function to compute stock attributes
def get_common_features():
  def stock_prec_dev(stock_symbol):
      #stock_symbol = 'ULTRACEMCO.NS'
      short_window = 20
      mid_window = 50
      long_window = 200
      moving_avg = 'SMA'

      # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
      start = datetime.datetime(*map(int, start_date.split('-')))
      end = datetime.datetime(*map(int, end_date.split('-')))
      stock_df = yfin.Ticker(stock_symbol).history(period='5y', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
      stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = long_window, min_periods = 1).mean(),0)
      stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
      stock_df.dropna(axis = 0, inplace = True) # remove any null rows

      stock_df['Stock'] = stock_symbol

      stock_df['Close'] = round(stock_df['Close'],2)
      stock_df['Max'] = round(max(stock_df['Close']),2)
      stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start1_date)]
      stock_df['Min'] = round(min(stock_df['Close']),2)
      stock_df['Gained%'] = round((stock_df['Close'] - stock_df['Min'])*100/stock_df['Min'],2)
      stock_df.ta.rsi(append=True)
      #stock_df.rename(columns={'RSI_14':'RSI'}, inplace=True)
      stock_df['Close'] = round(stock_df['Close'],0)
      stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
      stock_df.drop(['Open', 'Low', 'High', 'Volume'
                    ], axis=1, inplace=True)

      return stock_df

  df_prec_dev = pd.DataFrame()

  #tmp = stock_prec_dev('ULTRACEMCO.NS')
  #df_perform = df_perform.append(tmp)
  for stock_id in all_prospects:
      #print(stock_id)
      current_pe, current_roce, current_roe = get_current_features(stock_id)
      stock_id = stock_id.upper() + '.NS'
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      tmp = tmp.tail(1)
      tmp['Curr_PE'] = current_pe
      tmp['Curr_ROCE%'] = current_roce
      tmp['Curr_ROE%'] = current_roe
      df_prec_dev = df_prec_dev.append(tmp)
  return df_prec_dev, stock_prec_dev

df_common_features, stock_prec_dev = get_common_features()

In [10]:
# nifty in last 5 weeks
stock_prec_dev('^NSEI')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14
Date,,,,,,,,
2023-09-21 00:00:00+05:30,19742.0,18467.0,6.91,^NSEI,20192.35,16945.05,16.51,51.0
2023-09-28 00:00:00+05:30,19524.0,18492.0,5.58,^NSEI,20192.35,16945.05,15.22,42.0
2023-10-06 00:00:00+05:30,19654.0,18518.0,6.13,^NSEI,20192.35,16945.05,15.98,50.0
2023-10-13 00:00:00+05:30,19751.0,18554.0,6.45,^NSEI,20192.35,16945.05,16.56,53.0
2023-10-20 00:00:00+05:30,19537.0,18594.0,5.07,^NSEI,20192.35,16945.05,15.29,44.0


In [11]:
stock_ids = df_common_features['Stock'].values
df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)

In [12]:
df_prospects = pd.merge(df_common_features, df_mypf, on='Stock')

In [13]:
#df_common_features['Discount%PE'] = round((df_common_features['MedianPE']-df_common_features['Curr_PE'])*100/(df_common_features['MedianPE']),0)
#df_prospects['Target'] = np.where(df_prospects['Strategy']== 'BTT', df_prospects["Target"], df_prospects['Max'])
df_prospects['Upside%Target'] = round((df_prospects['Max']-df_prospects['Close'])*100/(df_prospects['Close']),0)
df_prospects.drop([ '200_SMA', 'Symbol', 'Target', 'Max', 'Min', 'Close'], axis=1, inplace=True)
df_prospects = df_prospects.set_index('Stock')

In [14]:
# top 10 prospects to invest from BTT stocks
df_prospects[df_prospects['Strategy'] == 'BTT'].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
BLUESTARCO.NS,26.64,67.03,72.0,62.90,24.4,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,60.0
UPL.NS,-9.30,4.74,44.0,16.80,14.3,13.40,AR,BTT,1,0,0.0,NaN,NaN,L,NaN,35.0
PNB.NS,27.19,87.10,45.0,18.20,4.1,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,32.0
AKZOINDIA.NS,1.67,19.73,34.0,30.50,34.7,26.10,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,18.0
ASIANPAINT.NS,0.95,15.27,34.0,63.70,34.4,27.70,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,14.0
MOTILALOFS.NS,36.88,80.75,63.0,10.40,12.9,15.60,V40N,BTT,1,1,0.0,NaN,0.0,L,FINANCE,10.0
NMDC.NS,34.40,66.70,61.0,9.37,30.2,23.00,VR,BTT,1,0,1.0,1.0,1.0,M,MINING,10.0
ABBOTINDIA.NS,1.80,25.04,43.0,46.20,41.1,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,9.0
BAJAJHIND.NS,57.94,164.53,58.0,1000.00,1000.0,1000.00,VR,BTT,1,0,1.0,1.0,1.0,M,SUGAR,8.0


In [15]:
# prospects to invest from star + V40/V40N/VR + below 200 DMA
df_prospects[(df_prospects['Criteria'].notnull()) & (df_prospects['StarStock'] == 1) & (df_prospects['Dev%_200'] < 5)].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
VINATIORGA.NS,-3.76,4.85,38.0,41.8,30.40,22.6,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,30.0
DABUR.NS,-5.15,3.41,24.0,53.6,22.70,19.5,V40,NaN,1,1,0.0,1.0,1.0,H,FMCG,24.0
AKZOINDIA.NS,1.67,19.73,34.0,30.5,34.70,26.1,V40,BTT,1,1,1.0,1.0,NaN,M,NaN,18.0
ASIANPAINT.NS,0.95,15.27,34.0,63.7,34.40,27.7,V40,BTT,1,1,1.0,1.0,1.0,H,PAINTS,14.0
ABBOTINDIA.NS,1.80,25.04,43.0,46.2,41.10,31.6,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,9.0
MARICO.NS,2.84,15.55,39.0,51.6,41.90,35.8,V40,NaN,1,1,0.0,NaN,1.0,M,FMCG,9.0
ICICIBANK.NS,2.74,16.04,38.0,17.5,6.32,17.2,V40,NaN,1,1,0.0,NaN,1.0,H,BANKS,6.0


In [16]:
# top 10 prospects to invest from star stocks
df_prospects[df_prospects['StarStock'] == 1].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ADANIGREEN.NS,-7.70,102.90,39.0,121.00,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,217.0
BLUESTARCO.NS,26.64,67.03,72.0,62.90,24.40,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,60.0
IGL.NS,-9.33,11.83,29.0,17.10,26.60,20.50,NaN,NaN,1,1,0.0,0.0,NaN,L,NaN,37.0
KANSAINER.NS,10.10,29.90,46.0,48.30,14.50,10.70,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,36.0
PNB.NS,27.19,87.10,45.0,18.20,4.10,3.34,VR,BTT,1,1,1.0,1.0,1.0,M,BANKS,32.0
VINATIORGA.NS,-3.76,4.85,38.0,41.80,30.40,22.60,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,30.0
MUTHOOTFIN.NS,13.30,42.80,58.0,13.50,12.10,17.90,NaN,NaN,1,1,0.0,NaN,0.0,L,FINANCE,29.0
BPCL.NS,-0.81,18.15,48.0,3.75,6.86,6.34,NaN,NaN,1,1,0.0,1.0,NaN,L,NaN,26.0
DABUR.NS,-5.15,3.41,24.0,53.60,22.70,19.50,V40,NaN,1,1,0.0,1.0,1.0,H,FMCG,24.0


In [17]:
# top 10 prospects to invest from  V40/V40N/VR
df_prospects[df_prospects['Criteria'].notnull()].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
JCHAC.NS,5.30,26.14,44.0,1000.0,1000.0,1000.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,171.0
SYMPHONY.NS,-9.49,3.83,39.0,53.0,14.8,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,72.0
SFL.NS,1.00,20.28,57.0,59.1,13.6,12.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,72.0
LALPATHLAB.NS,19.77,43.66,64.0,80.2,18.2,15.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,64.0
DREAMFOLKS.NS,-1.82,40.37,48.0,35.9,76.8,60.20,AR,ATH,0,0,0.0,1.0,0.0,L,MISC,64.0
BLUESTARCO.NS,26.64,67.03,72.0,62.9,24.4,23.40,AR,BTT,1,1,1.0,1.0,1.0,H,AC,60.0
PFIZER.NS,1.94,14.39,52.0,117.0,27.2,19.80,V40,NaN,1,0,0.0,NaN,0.0,L,PHARMA,53.0
TTKPRESTIG.NS,7.73,24.43,63.0,45.5,18.1,13.80,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,43.0
KANSAINER.NS,10.10,29.90,46.0,48.3,14.5,10.70,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,36.0


In [18]:
# top 10 prospects to invest below 200 DMA
df_prospects[df_prospects['Dev%_200'] < 0].sort_values(by = 'Upside%Target', ascending=False).head(10)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ADANIGREEN.NS,-7.70,102.90,39.0,121.0,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,217.0
ALKYLAMINE.NS,-9.04,4.75,38.0,60.4,41.30,41.40,NaN,NaN,0,0,1.0,1.0,0.0,M,CHEMICALS,104.0
BIOCON.NS,-3.87,18.94,24.0,39.0,5.92,4.78,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,104.0
SYMPHONY.NS,-9.49,3.83,39.0,53.0,14.80,13.50,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,72.0
DREAMFOLKS.NS,-1.82,40.37,48.0,35.9,76.80,60.20,AR,ATH,0,0,0.0,1.0,0.0,L,MISC,64.0
VOLTAS.NS,-0.02,11.35,36.0,89.1,9.67,4.42,NaN,NaN,1,0,0.0,NaN,0.0,L,AC,60.0
IGL.NS,-9.33,11.83,29.0,17.1,26.60,20.50,NaN,NaN,1,1,0.0,0.0,NaN,L,NaN,37.0
UPL.NS,-9.30,4.74,44.0,16.8,14.30,13.40,AR,BTT,1,0,0.0,NaN,NaN,L,NaN,35.0
VEDL.NS,-13.07,7.22,44.0,11.2,21.20,20.40,NaN,NaN,1,0,0.0,NaN,NaN,L,NaN,32.0


In [19]:
# star + v40/v40n prospects (for breakout)

tmp_df = df_prospects[df_prospects['StarStock'] == 1]
tmp_df = tmp_df[(tmp_df['Criteria'] == 'V40') | (tmp_df['Criteria'] == 'V40N')]
tmp_df = tmp_df[(tmp_df['Upside%Target'] < 5)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%Target', ascending=True)

qualified stocks: 9


,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
ANGELONE.NS,62.51,139.83,80.0,19.5,44.00,47.1,V40N,NaN,1,1,1.0,1.0,NaN,M,FINANCE,-0.0
BAJAJ-AUTO.NS,28.79,64.76,77.0,22.7,26.20,20.2,V40,NaN,1,1,0.0,1.0,NaN,L,NaN,0.0
COLPAL.NS,23.68,46.95,65.0,132.0,179.00,158.0,V40,BTT,1,1,0.0,1.0,1.0,H,FMCG,-0.0
EQUITASBNK.NS,29.81,115.92,68.0,14.8,8.04,12.2,V40N,NaN,1,1,1.0,NaN,1.0,M,BANKS,-0.0
ASTRAZEN.NS,26.91,51.08,56.0,70.5,30.90,23.0,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,2.0
TITAN.NS,17.08,42.42,54.0,90.2,25.10,30.8,V40,NaN,1,1,0.0,NaN,1.0,H,JEWELLERY,2.0
CAPLIPOINT.NS,33.78,85.28,55.0,20.7,26.30,22.3,V40N,NaN,1,1,1.0,1.0,NaN,M,NaN,3.0
HCLTECH.NS,12.83,29.10,57.0,22.0,28.30,23.0,V40,NaN,1,1,0.0,NaN,1.0,H,IT,3.0
MCDOWELL-N.NS,17.73,41.47,51.0,61.5,20.00,17.5,V40N,NaN,1,1,0.0,NaN,0.0,L,BREWERIES,4.0


In [20]:
# prospects for interactive analysis

data_table.DataTable(df_prospects, include_index=True)

,Dev%_200,Gained%,RSI_14,Curr_PE,Curr_ROCE%,Curr_ROE%,Criteria,Strategy,LatestQtr,StarStock,BizCheck,FinCheck,TechCheck,Conviction,Category,Upside%Target
Stock,,,,,,,,,,,,,,,,
3MINDIA.NS,17.16,41.79,44.0,68.70,31.40,23.40,V40N,NaN,1,1,1.0,NaN,NaN,M,NaN,9.0
ABBOTINDIA.NS,1.80,25.04,43.0,46.20,41.10,31.60,V40,BTT,1,1,1.0,1.0,NaN,M,PHARMA,9.0
ACC.NS,4.05,23.69,45.0,43.50,10.00,7.06,NaN,NaN,0,0,0.0,1.0,0.0,L,CEMENT,38.0
ADANIENT.NS,2.39,100.87,40.0,98.10,9.49,9.63,NaN,NaN,1,0,0.0,1.0,1.0,M,NaN,74.0
ADANIGREEN.NS,-7.70,102.90,39.0,121.00,7.81,22.80,NaN,NaN,1,1,0.0,1.0,0.0,L,NaN,217.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VIPIND.NS,3.20,18.90,46.0,56.40,27.80,29.00,V40N,NaN,1,0,1.0,NaN,NaN,M,NaN,13.0
VOLTAS.NS,-0.02,11.35,36.0,89.10,9.67,4.42,NaN,NaN,1,0,0.0,NaN,0.0,L,AC,60.0
WSTCSTPAPR.NS,36.66,63.77,61.0,4.89,52.90,44.80,VR,ATH,0,0,0.0,NaN,NaN,L,PAPER,2.0


In [21]:
# save the output
if gen_output == 1:
  from google.colab import drive
  drive.mount('/content/drive')
  df_prec_dev = df_prec_dev.reset_index()
  df_prec_dev.to_csv('/content/drive/My Drive/data/stocks/myProspects-Upside-Analysis-ATH.csv', index=False)

In [22]:
# stock in last 5 weeks
#stock_ids = ['ICICIGI','DABUR','MOTILALOFS','JPPOWER','EQUITAS','GLAND','JMFINANCIL','LUXIND','UJJIVANSFB','VAIBHAVGBL','VALIANTORG']
stock_prec_dev('BERGEPAINT.NS')[-21::5]

,Close,200_SMA,Dev%_200,Stock,Max,Min,Gained%,RSI_14
Date,,,,,,,,
2023-09-21 00:00:00+05:30,628.0,521.0,20.48,BERGEPAINT.NS,710.11,447.11,40.39,71.0
2023-09-28 00:00:00+05:30,570.0,524.0,8.69,BERGEPAINT.NS,710.11,447.11,27.38,40.0
2023-10-06 00:00:00+05:30,565.0,525.0,7.69,BERGEPAINT.NS,710.11,447.11,26.45,40.0
2023-10-13 00:00:00+05:30,568.0,527.0,7.83,BERGEPAINT.NS,710.11,447.11,27.09,43.0
2023-10-20 00:00:00+05:30,580.0,529.0,9.72,BERGEPAINT.NS,710.11,447.11,29.81,51.0
